# 概述

[Baidu ElasticSearch](https://cloud.baidu.com/doc/BES/index.html?from=productToDoc)（以下简称 BES）是一款100%兼容开源的分布式检索分析服务。为结构化/非结构化数据提供低成本、高性能及可靠性的检索、分析平台级产品服务。向量能力方面，支持多种索引类型和相似度距离算法。
目前，BES已经和文心一言大模型联合打造方案，也广泛应用于推荐、计算机视觉、智能问答等领域；同时，BES的向量检索和存储的能力也集成进入Langchain。
本文主要介绍基于Langchain的框架，结合BES的向量数据库的能力，对接千帆平台的模型管理和应用接入的能力，从而构建一个RAG的知识问答场景。

## 准备工作
用户需要保证python的版本大于等于3.9，且需要安装langchain，qianfan sdk的包。

In [1]:
!pip install langchain==0.332
!pip install elasticsearch==7.11.0
!pip install qianfan 
!pip install pdfplumber

ERROR: Could not find a version that satisfies the requirement langchain==0.332 (from versions: 0.0.1, 0.0.2, 0.0.3, 0.0.4, 0.0.5, 0.0.6, 0.0.7, 0.0.8, 0.0.9, 0.0.10, 0.0.11, 0.0.12, 0.0.13, 0.0.14, 0.0.15, 0.0.16, 0.0.17, 0.0.18, 0.0.19, 0.0.20, 0.0.21, 0.0.22, 0.0.23, 0.0.24, 0.0.25, 0.0.26, 0.0.27, 0.0.28, 0.0.29, 0.0.30, 0.0.31, 0.0.32, 0.0.33, 0.0.34, 0.0.35, 0.0.36, 0.0.37, 0.0.38, 0.0.39, 0.0.40, 0.0.41, 0.0.42, 0.0.43, 0.0.44, 0.0.45, 0.0.46, 0.0.47, 0.0.48, 0.0.49, 0.0.50, 0.0.51, 0.0.52, 0.0.53, 0.0.54, 0.0.55, 0.0.56, 0.0.57, 0.0.58, 0.0.59, 0.0.60, 0.0.61, 0.0.63, 0.0.64, 0.0.65, 0.0.66, 0.0.67, 0.0.68, 0.0.69, 0.0.70, 0.0.71, 0.0.72, 0.0.73, 0.0.74, 0.0.75, 0.0.76, 0.0.77, 0.0.78, 0.0.79, 0.0.80, 0.0.81, 0.0.82, 0.0.83, 0.0.84, 0.0.85, 0.0.86, 0.0.87, 0.0.88, 0.0.89, 0.0.90, 0.0.91, 0.0.92, 0.0.93, 0.0.94, 0.0.95, 0.0.96, 0.0.97, 0.0.98, 0.0.99rc0, 0.0.99, 0.0.100, 0.0.101rc0, 0.0.101, 0.0.102rc0, 0.0.102, 0.0.103, 0.0.104, 0.0.105, 0.0.106, 0.0.107, 0.0.108, 0.0.109, 0.0.

## 系统设置
1. 需要在千帆大模型平台创建应用，获得接入的ak、sk
2. 在BES的产品界面上创建一个BES集群，详见(https://cloud.baidu.com/doc/BES/s/0jwvyk4tv)

In [1]:
import os

os.environ['QIANFAN_AK'] = "your_qianfan_ak"
os.environ['QIANFAN_SK'] = "your_qianfan_sk"

## 开发过程
### 文档加载、切分

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PDFPlumberLoader


loader = PDFPlumberLoader("/Users/zhonghanjun/pywp/bce-qianfan-sdk/cookbook/RAG/baidu_elasticsearch/example_data/ai-paper.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 384, chunk_overlap = 0, separators=["\n\n", "\n", " ", "", "。", "，"])
all_splits = text_splitter.split_documents(documents)
all_splits

### 嵌入
创建对应的嵌入算法，这里采用千帆大模型平台的 `QianfanEmbeddingsEndpoint` 接口。

In [16]:
from langchain.embeddings import QianfanEmbeddingsEndpoint #sdk

embeddings = QianfanEmbeddingsEndpoint()
# embeddings-v1

### 向量检索和存储
引入创建的BES集群，用Langchain内部集成的`BESVectorStore`接口创建向量存储对象(详见:[BESVectorStore](https://python.langchain.com/docs/integrations/vectorstores/baiducloud_vector_search))

In [ ]:
from langchain.vectorstores import BESVectorStore

bes = BESVectorStore.from_documents(
    documents=documents,
    embedding=embeddings,
    bes_url="your bes cluster url",
    index_name="your vector index",
)
bes.client.indices.refresh(index="your vector index")

### 生成QA检索器
创建对应 ERNIE-Bot 或者 ERNIE-Bot-turbo 模型的 Langchain Chat Model用于进一步生成QA检索器。model 字段支持使用 `ERNIE-Bot-turbo` 或 `ERNIE-Bot`。这里我们使用 `ERNIE-Bot`

In [ ]:
from langchain.chat_models import QianfanChatEndpoint
from langchain.chains import RetrievalQA

retriever = bes.as_retriever(search_type="similarity_score_threshold", search_kwargs={'score_threshold': 0.0})
qianfan_chat_model = QianfanChatEndpoint(model="ERNIE-Bot")
# sdk prompt load from qianfan
qa = RetrievalQA.from_chain_type(llm=qianfan_chat_model, chain_type="refine", retriever=retriever, return_source_documents=True)


query = input("\n请输入问题: ")
res = qa(query)
answer, docs = res['result'], res['source_documents']

print("\n\n> 问题:")
print(query)
print("\n> 回答:")
print(answer)

In [ ]:
> 问题:
2021年12月,DeepMind做了什么?

> 回答:
根据上下文，2021年12月，DeepMind发布了Gopher模型。该模型具有2800亿参数。经过152个任务的评估，Gopher比当时最先进的语言模型提高了大约81%的性能，特别是在知识密集领域，如事实检测和常识上。
因此，正确答案是发布Gopher模型。